In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('data/game_logs.csv', low_memory = False)
df


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,...,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,...,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,...,Davy Force,6.0,mille105,Everett Mills,3.0,allid101,Doug Allison,2.0,hallg101,George Hall,7.0,leona101,Andy Leonard,4.0,braia102,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,4,54.0,D,NaN,NaN,NaN,RCK01,1000.0,140.0,610020003,010020100,49.0,11.0,1.0,1.0,...,Bob Addy,4.0,fishc102,Cherokee Fisher,1.0,hasts101,Scott Hastings,8.0,ham-r101,Ralph Ham,5.0,ansoc101,Cap Anson,2.0,sagep101,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,14,54.0,D,NaN,NaN,NaN,CHI01,5000.0,150.0,101403111,077000000,46.0,15.0,2.0,1.0,...,Marshall King,8.0,hodec101,Charlie Hodes,2.0,woodj106,Jimmy Wood,4.0,simmj101,Joe Simmons,9.0,folet101,Tom Foley,7.0,duffe101,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,5,54.0,D,NaN,NaN,NaN,TRO01,3250.0,145.0,000002232,101003000,46.0,17.0,4.0,1.0,...,Mike McGeary,2.0,yorkt101,Tom York,8.0,mcmuj101,John McMullin,1.0,kings101,Steve King,7.0,beave101,Edward Beavens,4.0,bells101,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,4,60.0,D,NaN,NaN,NaN,DEN02,27762.0,203.0,0200000202,1100100010,39.0,10.0,4.0,1.0,...,David Dahl,7.0,arenn001,Nolan Arenado,5.0,gonzc001,Carlos Gonzalez,9.0,murpt002,Tom Murphy,2.0,pattj005,Jordan Patterson,3.0,valap001,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,5,51.0,D,NaN,NaN,NaN,PHI13,36935.0,159.0,000001100,00100031x,33.0,8.0,3.0,0.0,...,Jimmy Paredes,7.0,herro001,Odubel Herrera,8.0,franm004,Maikel Franco,5.0,howar001,Ryan Howard,3.0,ruppc001,Cameron Rupp,2.0,blana001,Andres Blanco,6.0,altha001,Aaron Altherr,9.0,eickj001,Jerad Eickhoff,1.0,NaN,Y
171904,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,7,51.0,D,NaN,NaN,NaN,SFO03,41445.0,184.0,000100000,23000002x,30.0,4.0,0.0,0.0,...,Brandon Belt,3.0,poseb001,Buster Posey,2.0,pench001,Hunter Pence,9.0,crawb001,Brandon Crawford,6.0,pagaa001,Angel Pagan,7.0,panij002,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,NaN,Y
171905,20161002,0,Sun,PIT,NL,162,SLN,NL,162,4,10,51.0,D,NaN,NaN,NaN,STL10,44615.0,192.0,000020200,00100360x,35.0,9.0,0.0,0.0,...,Aledmys Diaz,6.0,moliy001,Yadier Molina,2.0,piscs001,Stephen Piscotty,9.0,peraj001,Jhonny Peralta,5.0,mossb001,Brandon Moss,7.0,gyorj001,Jedd Gyorko,4.0,gricr001,Randal Grichuk,8.0,waina001,Adam Wainwright,1.0,NaN,Y


In [3]:
# Dropping redundant columns
dropping = []
for column in df.columns[7:]:
    if column.endswith('_def_pos'):
        dropping.append(column)
    elif '_name' in column:
        dropping.append(column)
    elif '_info' in column:
        dropping.append(column)
    elif 'lf_umpire_id' in column:
        dropping.append(column)
    elif 'rf_umpire_id' in column:
        dropping.append(column)
df = df.drop(columns = dropping)

In [4]:
# Removing the 2 forfeited games & column
df = df[df['forefeit'].isna()==True]
df = df.drop(columns = 'forefeit')

In [5]:
# Removing interrupted games & completion column
df = df[df['completion'].isna()==True]
df = df.drop(columns = 'completion')

In [6]:
# Remove protested games & column
df = df[df['protest'].isna()==True]
df = df.drop(columns = 'protest')

In [7]:
# Remove multi-headed games
df = df[df['number_of_game'] == 0]
df = df.drop(columns = 'number_of_game')

In [8]:
# Remove draws
df = df[df['h_score'] != df['v_score']]

In [9]:
# Random Weighted Imputing for day_night column
choice = sorted(df[df.day_night.isna() == False]['day_night'].unique())
prob = df[df.day_night != 0]['day_night'].value_counts(normalize=True).sort_index()
df['day_night'] = df.day_night.map(lambda x: np.random.choice(choice, 1, p = prob)[0] if x not in choice else x)


In [10]:
# Median Imputing for attendance & length_minutes missing values
df['attendance'] = df['attendance'].fillna(value= df['attendance'].median())
df['length_minutes'] = df['length_minutes'].fillna(value= df['length_minutes'].median())

In [11]:
# Reset Index & Assign Match ID
df.reset_index(inplace = True)
df.rename(columns = {'index': 'matchID'}, inplace = True)

In [12]:
# Categorize winning team for restructuring dataset
winning_team = []
for i in range(0,len(df)):
    if df['h_score'][i] > df['v_score'][i] == True:
        winning_team.append('Home')
    else:
        winning_team.append('Visitor')
df['winning_team'] = winning_team

In [13]:
# Define unique home/visitor stats
h_stats = []
for column in df.columns:
    if str(column).startswith('h_'):
        h_stats.append(column)

v_stats = []
for column in df.columns:
    if str(column).startswith('v_'):
        v_stats.append(column)

In [14]:
# Extracting only winning team and respective statistics
dfwin1 = df[df['winning_team'] == 'Home']
dfwin1 = dfwin1.drop(columns = v_stats, axis = 1)

dfwin2 = df[df['winning_team'] == 'Visitor']
dfwin2 = dfwin2.drop(columns = h_stats, axis = 1)


In [15]:
# Renaming h_ and v_ columns
for column in dfwin1.columns:
    if str(column).startswith('h_') == True:
        dfwin1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfwin2.columns:
    if str(column).startswith('v_') == True:
        dfwin2.rename(columns = {column: column[2:]}, inplace = True)

In [18]:
# Merging both win df
dfwin = pd.concat([dfwin1, dfwin2], sort = False) 

In [19]:
# Assigning boolean target variable "winner"
dfwin["winner"] = np.nan
dfwin['winner'].fillna(value = 1, inplace = True)

In [20]:
# Extracting only losing team and respective statistics
dfloss1 = df[df['winning_team'] == 'Home']
dfloss1 = dfloss1.drop(columns = h_stats, axis = 1)

dfloss2 = df[df['winning_team'] == 'Visitor']
dfloss2 = dfloss2.drop(columns = v_stats, axis = 1)

In [21]:
# Renaming h_ and v_ columns
for column in dfloss1.columns:
    if str(column).startswith('v_') == True:
        dfloss1.rename(columns = {column: column[2:]}, inplace = True)
        
for column in dfloss2.columns:
    if str(column).startswith('h_') == True:
        dfloss2.rename(columns = {column: column[2:]}, inplace = True)


In [22]:
# Merging both loss df
dfloss = pd.concat([dfloss1, dfloss2], sort = False)

In [23]:
# Assigning boolean target variable "winner"
dfloss["winner"] = np.nan
dfloss['winner'].fillna(value = 0, inplace = True)

In [24]:
# Combine both win and loss dataframe
df = pd.concat([dfwin, dfloss], sort = False)

In [25]:
df.to_csv('df.csv')